### EXIF from DJI

Test snippets for reading EXIF from a directory of images. This version also reads the DJI-specific metadata from the XMP portion of the record.
Read all of the files in a directory and generate a .csv file with name, lat, lon, altitude

Part of this was adapted from https://www.codingforentrepreneurs.com/blog/extract-gps-exif-images-python/

In [1]:
import sys
#import glob
import os
#from os.path import isfile, join, basename
#from os import listdir
import numpy as np

# EXIF Reader
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

In [2]:
def get_exif_data(imgpath):
    """
    Returns a dictionary from the exif data of an PIL Image item. Also converts the GPS Tags
    https://www.codingforentrepreneurs.com/blog/extract-gps-exif-images-python/
    """    
    exif_data = {}
    image=Image.open(imgpath)
    info = image._getexif()
    if info:
        for tag, value in info.items():
            decoded = TAGS.get(tag, tag)
            if decoded == "GPSInfo":
                gps_data = {}
                for t in value:
                    sub_decoded = GPSTAGS.get(t, t)
                    gps_data[sub_decoded] = value[t]

                exif_data[decoded] = gps_data
            else:
                exif_data[decoded] = value
    return exif_data

In [3]:
def latlon_from_gps_data(gps_data):
    try: 
        v = gps_data['GPSLatitude']
        lat = float(v[0][0])/float(v[0][1])+\
        (1./60.)*float(v[1][0])/float(v[1][1])+\
        (1./3600.)*float(v[2][0])/float(v[2][1])
        if gps_data['GPSLatitudeRef']=='S':
            lat = -1.*lat

        v = gps_data['GPSLongitude'] 
        lon = float(v[0][0])/float(v[0][1])+\
        (1./60.)*float(v[1][0])/float(v[1][1])+\
        (1./3600.)*float(v[2][0])/float(v[2][1])
        if gps_data['GPSLongitudeRef']=='W':
            lon = -1.*lon           
        try:
            v = gps_data['GPSAltitude']
            alt = float(v[0])/float(v[1])
        except:
            alt=np.nap
            
        return lat, lon, alt
    
    except:
        return np.nan, np.nan, np.nan

def get_gps_data(i):
    info = i._getexif()
    exif_data={}
    lat = np.nan
    lon = np.nan
    alt = np.nan
    if info:
        for tag, value in info.items():
            decoded = TAGS.get(tag, tag)
            if decoded == "GPSInfo":
                gps_data = {}
                for t in value:
                    sub_decoded = GPSTAGS.get(t, t)
                    gps_data[sub_decoded] = value[t]
                lat,lon,alt=latlon_from_gps_data(gps_data)
    else:
        print('Could not find EXIF data in ',filepath)
        
    return lat,lon,alt

In [4]:
#dn = 'C:/crs/proj/2015_Sandwich/2016-02-11_PT_images/down/'
path = r"."
file = 'DJI_0012.JPG'

i = Image.open(os.path.join(path,file))
if i:
    lat,lon,alt = get_gps_data(i)
    print('{0},{1:.8f},{2:.8f},{3:.3f}'.format(file,lat,lon,alt))

else:
    print('Could not open ',dn+file)
    
exif_info = get_exif_data(os.path.join(path,file))
print(exif_info.keys())

DJI_0012.JPG,41.76829936,-70.48680300,154.170
dict_keys(['GPSInfo', 'ResolutionUnit', 'ExifOffset', 'ImageDescription', 'Make', 'Model', 'Software', 'Orientation', 'DateTime', 'YCbCrPositioning', 'XResolution', 'YResolution', 'XPComment', 'XPKeywords', 'ExifVersion', 'ComponentsConfiguration', 'CompressedBitsPerPixel', 'DateTimeOriginal', 'DateTimeDigitized', 'ShutterSpeedValue', 'ApertureValue', 'ExposureBiasValue', 'MaxApertureValue', 'SubjectDistance', 'MeteringMode', 'LightSource', 'Flash', 'FocalLength', 'ColorSpace', 'ExifImageWidth', 'ExifImageHeight', 'Contrast', 'Saturation', 'Sharpness', 'DeviceSettingDescription', 'ExposureIndex', 'FileSource', 'ExposureTime', 'ExifInteroperabilityOffset', 'FNumber', 'SceneType', 'ExposureProgram', 'CustomRendered', 'ISOSpeedRatings', 'ExposureMode', 'FlashPixVersion', 'WhiteBalance', 'DigitalZoomRatio', 'FocalLengthIn35mmFilm', 'SceneCaptureType', 'GainControl', 'SubjectDistanceRange', 'MakerNote'])


In [5]:
exif_info

{'ApertureValue': (200, 100),
 'ColorSpace': 1,
 'ComponentsConfiguration': b'\x00\x03\x02\x01',
 'CompressedBitsPerPixel': (4771690, 1500000),
 'Contrast': 0,
 'CustomRendered': 0,
 'DateTime': '2017:01:09 12:48:27',
 'DateTimeDigitized': '2017:01:09 12:48:27',
 'DateTimeOriginal': '2017:01:09 12:48:27',
 'DeviceSettingDescription': b'\x00\x00\x00\x00',
 'DigitalZoomRatio': (0, 0),
 'ExifImageHeight': 3000,
 'ExifImageWidth': 4000,
 'ExifInteroperabilityOffset': 656,
 'ExifOffset': 182,
 'ExifVersion': b'0230',
 'ExposureBiasValue': (0, 32),
 'ExposureIndex': (0, 0),
 'ExposureMode': 0,
 'ExposureProgram': 2,
 'ExposureTime': (460, 1000000),
 'FNumber': (280, 100),
 'FileSource': b'\x03',
 'Flash': 32,
 'FlashPixVersion': b'0010',
 'FocalLength': (361, 100),
 'FocalLengthIn35mmFilm': 20,
 'GPSInfo': {'GPSAltitude': (154170, 1000),
  'GPSAltitudeRef': b'\x01',
  'GPSLatitude': ((41, 1), (46, 1), (58777, 10000)),
  'GPSLatitudeRef': 'N',
  'GPSLongitude': ((70, 1), (29, 1), (124908, 100

In [6]:
fd = open(file,'rb')
d= fd.read()
xmp_start = d.find(b'<x:xmpmeta')
xmp_end = d.find(b'</x:xmpmeta')
xmp_b = d[xmp_start:xmp_end+12]
# print(type(xmp_b))
# print(xmp_b)
xmp_str = xmp_b.decode()
print(xmp_str)

<x:xmpmeta xmlns:x="adobe:ns:meta/">
 <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
  <rdf:Description rdf:about="DJI Meta Data"
    xmlns:tiff="http://ns.adobe.com/tiff/1.0/"
    xmlns:exif="http://ns.adobe.com/exif/1.0/"
    xmlns:xmp="http://ns.adobe.com/xap/1.0/"
    xmlns:xmpMM="http://ns.adobe.com/xap/1.0/mm/"
    xmlns:dc="http://purl.org/dc/elements/1.1/"
    xmlns:crs="http://ns.adobe.com/camera-raw-settings/1.0/"
    xmlns:drone-dji="http://www.dji.com/drone-dji/1.0/"
   xmp:ModifyDate="2017-01-09"
   xmp:CreateDate="2017-01-09"
   tiff:Make="DJI"
   tiff:Model="FC300X"
   dc:format="image/jpeg"
   drone-dji:AbsoluteAltitude="-154.17"
   drone-dji:RelativeAltitude="+70.40"
   drone-dji:GimbalRollDegree="+0.00"
   drone-dji:GimbalYawDegree="+126.00"
   drone-dji:GimbalPitchDegree="-90.00"
   drone-dji:FlightRollDegree="+1.40"
   drone-dji:FlightYawDegree="+122.70"
   drone-dji:FlightPitchDegree="-17.60"
   crs:Version="7.0"
   crs:HasSettings="False"
   crs

In [7]:
# List of DJI metadata labels we might want to grab
djimeta=["AbsoluteAltitude","RelativeAltitude","GimbalRollDegree","GimbalYawDegree",\
         "GimbalPitchDegree","FlightRollDegree","FlightYawDegree","FlightPitchDegree"]
# parse the XMP string to grab the values
xmp_dict={}
for m in djimeta:
    istart = xmp_str.find(m)
    ss=xmp_str[istart:istart+len(m)+10]
#     print(ss)
#     print( ss.split('"'))
    val = float(ss.split('"')[1])
    xmp_dict.update({m : val})
#    print(m,val)

print(xmp_dict)

{'AbsoluteAltitude': -154.17, 'RelativeAltitude': 70.4, 'GimbalRollDegree': 0.0, 'GimbalYawDegree': 126.0, 'GimbalPitchDegree': -90.0, 'FlightRollDegree': 1.4, 'FlightYawDegree': 122.7, 'FlightPitchDegree': -17.6}


In [8]:
def get_dji_meta( filepath ):
    """
    Returns a dict with DJI-specific metadata stored in the XMB portion of the image
    """
    
    # list of metadata tags
    djimeta=["AbsoluteAltitude","RelativeAltitude","GimbalRollDegree","GimbalYawDegree",\
         "GimbalPitchDegree","FlightRollDegree","FlightYawDegree","FlightPitchDegree"]
    
    # read file in binary format and look for XMP metadata portion
    fd = open(filepath,'rb')
    d= fd.read()
    xmp_start = d.find(b'<x:xmpmeta')
    xmp_end = d.find(b'</x:xmpmeta')

    # convert bytes to string
    xmp_b = d[xmp_start:xmp_end+12]
    xmp_str = xmp_b.decode()
    
    fd.close()
    
    # parse the XMP string to grab the values
    xmp_dict={}
    for m in djimeta:
        istart = xmp_str.find(m)
        ss=xmp_str[istart:istart+len(m)+10]
        val = float(ss.split('"')[1])
        xmp_dict.update({m : val})
        
    return xmp_dict

In [9]:
# test the function
file = 'DJI_0012.JPG' 
xmp_dict = get_dji_meta( file )
print(xmp_dict)
altr = xmp_dict['RelativeAltitude']
print(altr)

{'AbsoluteAltitude': -154.17, 'RelativeAltitude': 70.4, 'GimbalRollDegree': 0.0, 'GimbalYawDegree': 126.0, 'GimbalPitchDegree': -90.0, 'FlightRollDegree': 1.4, 'FlightYawDegree': 122.7, 'FlightPitchDegree': -17.6}
70.4


In [10]:
# test reading all of the data from a directory
